In [2]:
import zipfile
import pandas as pd
import numpy as np
import os
import glob
import shutil

In [ ]:
zip_path = 'telecom1000k_wo_result.zip'
extract_to = 'extracted_files'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

for root, dirs, files in os.walk(extract_to):
    for file in files:
        if ':' in file:
            new_name = file.replace(':', '-')
            os.rename(os.path.join(root, file), os.path.join(root, new_name))
            print(f'Renamed {file} to {new_name}')

In [ ]:
shutil.make_archive(zip_path, 'zip', extract_to)
shutil.rmtree(extract_to)

In [ ]:
with zipfile.ZipFile('extracted_files.zip', 'r') as zip_file:
    zip_file.extractall()

In [2]:
f_path = 'telecom1000k'
f_pattern = 'psx_*.csv'
f_paths = glob.glob(os.path.join(f_path, f_pattern))

output_f = 'comb_psx_data.csv'

with open(output_f, 'w', newline='') as f_out:
    first_file = True
    for file in f_paths:
        for chunk in pd.read_csv(file, chunksize=100000, dtype={'EndSession': 'object'}):
            chunk.to_csv(f_out, index=False, header=first_file, mode='a')
            first_file = False

In [3]:
data = pd.read_csv('comb_psx_data.csv')
data.head()

,IdSession,IdPSX,IdSubscriber,StartSession,EndSession,Duartion,UpTx,DownTx
0,8913,3,362086,31-12-2023 12:50:15,NaN,57585,1718690230,2253784059
1,8914,3,277559,31-12-2023 21:12:10,NaN,27470,4902814031,10010727635
2,31813,3,495588,01-01-2024 02:53:05,NaN,7015,242985980,296186764
3,8915,3,551729,31-12-2023 15:00:32,NaN,49768,9824381568,24676998355
4,8916,3,777373,31-12-2023 20:13:31,NaN,30989,240839066,306323953


In [4]:
f_path = 'telecom1000k'
f_pattern = 'psx_*.txt'
f_paths = glob.glob(os.path.join(f_path, f_pattern))

output_f = 'comb_psx_data1.csv'

with open(output_f, 'w', newline='') as f_out:
    first_file = True
    for file in f_paths:
        for chunk in pd.read_csv(file, chunksize=100000, dtype={'EndSession': 'object'}):
            chunk.to_csv(f_out, index=False, header=first_file, mode='a')
            first_file = False

In [5]:
data1 = pd.read_csv('comb_psx_data1.csv', sep = '|')
data1.head()

,IdSession,IdPSX,IdSubscriber,StartSession,EndSession,Duartion,UpTx,DownTx
0,151369,1,417967,03/01/2024 05:58:05,NaN,27715,10622352648,12825148352
1,148131,1,298042,03/01/2024 04:26:57,NaN,33183,941365040,1154043248
2,149233,1,481882,03/01/2024 04:52:09,NaN,31671,2158345192,3218451064
3,149234,1,292872,03/01/2024 04:53:51,NaN,31569,98246804216,193174707312
4,162425,1,687487,03/01/2024 11:19:37,NaN,8423,4777507696,5211655008


In [6]:
subscriber_data = pd.read_csv('telecom1000k/subscribers.csv')
client_data = pd.read_parquet('telecom1000k/client.parquet')
physical_data = pd.read_parquet('telecom1000k/physical.parquet')
company_data = pd.read_parquet('telecom1000k/company.parquet')

In [7]:
data = data.merge(subscriber_data, left_on='IdSubscriber', right_on='IdOnPSX')

In [8]:
del subscriber_data

In [9]:
data['raznic_uptx_downtx'] = data['UpTx'] - data['DownTx']
data['Traffic'] = data['UpTx'] + data['DownTx']

In [10]:
lower_percentile = data['raznic_uptx_downtx'].quantile(0.00001)
upper_percentile = data['raznic_uptx_downtx'].quantile(0.99)

outliers = data[(data['raznic_uptx_downtx'] < lower_percentile) | 
               (data['raznic_uptx_downtx'] > upper_percentile)]

print(outliers[['IdOnPSX', 'raznic_uptx_downtx']])

          IdOnPSX  raznic_uptx_downtx
1773       259766          1883538091
2338       931148           772841489
2389       361790          1100951596
7057        81437         24185272189
8383       967336          1400873023
...           ...                 ...
57941150   366776          3417396101
57941461   838729           899072290
57941498   831776          1468602682
57941520   378604          1388338856
57941667   592850          5249216551

[579998 rows x 2 columns]


In [11]:
new_data = outliers.copy()
del outliers

In [12]:
new_data = new_data.merge(client_data, left_on='IdClient', right_on='Id')
del client_data
new_data['StartSession'] = new_data['StartSession'].fillna('unkown')
new_data['TurnOn'] = new_data['Status'].map({'ON': True, 'OFF': False})
new_data['Type'] = np.where(
    new_data['IdClient'].isin(physical_data['Id']), 'P',
    np.where(new_data['IdClient'].isin(company_data['Id']), 'J', None)
)
del physical_data, company_data

In [13]:
new_anomaly_data_drop_dupli = new_data.drop_duplicates(
    subset=['IdOnPSX', 'IdClient', 'Type', 'IdPlan', 'TurnOn']
)
del new_data

In [22]:
new_anomaly_data_drop_dupli.shape

(2520, 19)

In [17]:
new_anomaly_data_drop_dupli.columns

Index(['IdSession', 'IdPSX', 'IdSubscriber', 'StartSession', 'EndSession',
       'Duartion', 'UpTx', 'DownTx', 'IdClient', 'IdOnPSX', 'Status',
       'raznic_uptx_downtx', 'Id', 'Contract', 'Documents', 'Email', 'IdPlan',
       'TurnOn', 'Type'],
      dtype='object')

In [14]:
del data

In [15]:
last_data = new_anomaly_data_drop_dupli[['IdSubscriber', 'IdClient', 'Type', 'IdPlan', 'TurnOn', 'Traffic']]

In [16]:
last_data['Hacked'] = True

<ipython-input-16-c92a42f1eed7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_data['Hacked'] = True


In [17]:
last_data.to_csv('last_data.csv', index=False)

Вторая половина

In [19]:
subscriber_data = pd.read_csv('telecom1000k/subscribers.csv')
client_data = pd.read_parquet('telecom1000k/client.parquet')
physical_data = pd.read_parquet('telecom1000k/physical.parquet')
company_data = pd.read_parquet('telecom1000k/company.parquet')

In [20]:
data1 = data1.merge(subscriber_data, left_on='IdSubscriber', right_on='IdOnPSX')

In [21]:
del subscriber_data

In [22]:
data1['raznic_uptx_downtx'] = data1['UpTx'] - data1['DownTx']
data1['Traffic'] = data1['UpTx'] + data1['DownTx']

In [24]:
lower_percentile = data1['raznic_uptx_downtx'].quantile(0.00001)
upper_percentile = data1['raznic_uptx_downtx'].quantile(0.99)

outliers = data1[(data1['raznic_uptx_downtx'] < lower_percentile) | 
               (data1['raznic_uptx_downtx'] > upper_percentile)]

print(outliers[['IdOnPSX', 'raznic_uptx_downtx']])

          IdOnPSX  raznic_uptx_downtx
20         415318         14034196928
42         537757          4782413296
79         345340          8893364840
201        926212          9623620160
292         56713          5394910000
...           ...                 ...
57589848    52983         16564706520
57589915   477910          4759393536
57590080   241579          6969240936
57590138   647504         11705025704
57590270   941051          6155163152

[576479 rows x 2 columns]


In [25]:
new_data = outliers.copy()
del outliers

In [26]:
new_data = new_data.merge(client_data, left_on='IdClient', right_on='Id')
del client_data
new_data['StartSession'] = new_data['StartSession'].fillna('unkown')
new_data['TurnOn'] = new_data['Status'].map({'ON': True, 'OFF': False})
new_data['Type'] = np.where(
    new_data['IdClient'].isin(physical_data['Id']), 'P',
    np.where(new_data['IdClient'].isin(company_data['Id']), 'J', None)
)
del physical_data, company_data

In [27]:
new_anomaly_data_drop_dupli = new_data.drop_duplicates(
    subset=['IdOnPSX', 'IdClient', 'Type', 'IdPlan', 'TurnOn']
)
del new_data

In [28]:
new_anomaly_data_drop_dupli.shape

(2543, 19)

In [29]:
last_data1 = new_anomaly_data_drop_dupli[['IdSubscriber', 'IdClient', 'Type', 'IdPlan', 'TurnOn', 'Traffic']]

In [30]:
last_data1['Hacked'] = True

<ipython-input-30-869c22b41e4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_data1['Hacked'] = True


In [31]:
last_data1.to_csv('last_data1.csv', index=False)

In [32]:
import gc
gc.collect()

0

In [36]:
df1 = pd.read_csv('last_data.csv')
df2 = pd.read_csv('last_data1.csv')

In [37]:
df = pd.concat([df1, df2], ignore_index=True)

In [39]:
df.shape

(5063, 6)

In [40]:
df_duplicates = df.drop_duplicates()

In [41]:
df_duplicates.shape

(4021, 6)

In [ ]:
df_duplicates['Traffic'] = df_duplicates['Traffic'] / 8192

In [6]:
df_duplicates.columns

Index(['IdSubscriber', 'IdClient', 'Type', 'IdPlan', 'TurnOn', 'Traffic',
       'Hacked'],
      dtype='object')

In [7]:
df_duplicates = df_duplicates.rename(columns={
    'IdSubscriber': 'Id',
    'IdClient': 'UID'
})

In [8]:
df_duplicates.to_csv('RESULT.txt', index=False, sep='\t')